In [38]:
import os
import gc
import cv2
import copy
import random
import numpy as np
import pandas as pd
from pathlib import Path
import typing as tp
from io import BytesIO
from PIL import Image
import h5py
import timm
from time import time
from tqdm.notebook import tqdm
import shutil

import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold

import torch
import torchvision
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.cuda import amp
from torch.utils.data import Dataset
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [39]:
class CFG:
    
    exp = 32
    
    is_kaggle = False
    is_infer = False
    if is_kaggle:
        OUTPUT_DIR = Path('/kaggle/input/isic2024-baseline')
        TRAIN_DIR = Path('/kaggle/input/isic-2024-challenge/train-image/image')
        TRAIN_HDF5 = Path('/kaggle/input/isic-2024-challenge/train-image.hdf5')
        TEST_HDF5 = Path('/kaggle/input/isic-2024-challenge/test-image.hdf5')
        TRAIN_META = Path('/kaggle/input/isic-2024-challenge/train-metadata.csv')
        TEST_META = Path('/kaggle/input/isic-2024-challenge/test-metadata.csv')
        SAMPLE_SUB = Path('/kaggle/input/isic-2024-challenge/sample_submission.csv')
        PRETRAINED_MODEL = ''
        
    else:
        OUTPUT_DIR = Path(f'/root/Development/Kaggle/ISIC2024/main/models/experiments/exp{exp}/outputs')
        OUTPUT_LOG = Path(f'/root/Development/Kaggle/ISIC2024/main/models/experiments/exp{exp}/log.txt')
        TRAIN_DIR = Path('/root/Development/Kaggle/ISIC2024/data/raw/train-image/image')
        TRAIN_HDF5 = Path('/root/Development/Kaggle/ISIC2024/data/raw/train-image.hdf5')
        TEST_HDF5 = Path('/root/Development/Kaggle/ISIC2024/data/raw/test-image.hdf5')
        TRAIN_META = Path('/root/Development/Kaggle/ISIC2024/data/raw/train-metadata.csv')
        TEST_META = Path('/root/Development/Kaggle/ISIC2024/data/raw/test-metadata.csv')
        SAMPLE_SUB = Path('/root/Development/Kaggle/ISIC2024/data/raw/sample_submission.csv')
        TRAIN_META_ADD = Path('/root/Development/Kaggle/ISIC2024/data/external/All/metadata.csv')
        TRAIN_HDF5_ADD = Path('/root/Development/Kaggle/ISIC2024/data/external/All/image.hdf5')
        TRAIN_HDF5_COMBINED = Path('/root/Development/Kaggle/ISIC2024/data/processed/train-image-combined.hdf5')
        PRETRAINED_MODEL = ''
        # PRETRAINED_MODEL = Path('/root/Development/Kaggle/ISIC2024/main/models/experiments/exp10/outputs/averaged_model.pth')
        
    
        
    max_epoch = 9
    n_folds = 5
    n_classes = 2
    random_seed = 42
    deterministic = True
    enable_amp = True
    view = True
    change_dataset = True
    standardization = True
    oversampling = False
    oversampling_percent = 3
    only_lesion_id = False
    alldata_isic_archive = True
    
    model_name = 'maxvit_rmlp_pico_rw_256.sw_in1k'
    output_dim_models = { 
        "resnet18.a1_in1k": 512,
        "efficientnet_b0.ra_in1k": 320,
        "tf_efficientnet_b0.ns_jft_in1k": 1280,
        # 'tf_efficientnet_b5.ns_jft_in1k': 1280,
        'tf_efficientnet_b3.ns_jft_in1k': 1536,
        'eva02_large_patch14_224.mim_in22k': 384,
        'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k': 384,
        'maxvit_tiny_tf_224.in1k': 256,
        'maxvit_rmlp_nano_rw_256.sw_in1k': 256,
        'maxvit_rmlp_pico_rw_256.sw_in1k': 256,
    }
    pretrained = True
    # img_size = 224
    img_size = 256
    # img_size = 384
    interpolation = cv2.INTER_LINEAR

    es_patience = 5 
    batch_size = 128
    
    # lr = 1.0e-03
    lr = 1.0e-04
    weight_decay = 1.0e-02 # default
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"

In [40]:

def F_rgb2hsv(rgb: torch.Tensor) -> torch.Tensor:
    cmax, cmax_idx = torch.max(rgb, dim=1, keepdim=True)
    cmin = torch.min(rgb, dim=1, keepdim=True)[0]
    delta = cmax - cmin
    hsv_h = torch.empty_like(rgb[:, 0:1, :, :])
    cmax_idx[delta == 0] = 3
    hsv_h[cmax_idx == 0] = (((rgb[:, 1:2] - rgb[:, 2:3]) / delta) % 6)[cmax_idx == 0]
    hsv_h[cmax_idx == 1] = (((rgb[:, 2:3] - rgb[:, 0:1]) / delta) + 2)[cmax_idx == 1]
    hsv_h[cmax_idx == 2] = (((rgb[:, 0:1] - rgb[:, 1:2]) / delta) + 4)[cmax_idx == 2]
    hsv_h[cmax_idx == 3] = 0.
    hsv_h /= 6.
    hsv_s = torch.where(cmax == 0, torch.tensor(0.).type_as(rgb), delta / cmax)
    hsv_v = cmax
    return torch.cat([hsv_h, hsv_s, hsv_v], dim=1)

# linear can treat any input size
class DynamicLinear(nn.Module):
    def __init__(self, out_size):
        super(DynamicLinear, self).__init__()
        self.out_size = out_size
        self.linear = None

    def forward(self, x):
        if self.linear is None:
            in_features = x.size(-1)
            self.linear = nn.Linear(in_features, self.out_size).to(x.device)
        
        return self.linear(x)


class timmModel(nn.Module):

    def __init__(
            self,
            model_name: str,
            pretrained: bool,
            in_channels: int,
            num_classes: int,
            is_training: bool=False
        ):
        super().__init__()
        if 'eva' in CFG.model_name:
            self.model = timm.create_model(
                model_name=model_name, 
                pretrained=pretrained, 
                in_chans=in_channels,
                num_classes=num_classes,
                # global_pool=''
            )
        else:
            self.model = timm.create_model(
                model_name=model_name, 
                pretrained=pretrained, 
                in_chans=in_channels,
                num_classes=num_classes,
                global_pool=''
            )
        
        self.output_type = ['infer', 'loss']
        self.is_training = is_training

        # model output dim
        dim = CFG.output_dim_models[CFG.model_name]
        
        self.dropout = nn.ModuleList([
            nn.Dropout(0.5) for i in range(5)
        ])
        # self.target=nn.Linear(dim, 1)
        self.target=DynamicLinear(out_size=1)
        # self.target_aux=nn.Linear(dim, 1)
        

    def forward(self, x, target=None):
        batch_size = len(x)
        
        x = torch.cat([x, F_rgb2hsv(x)],1) # (bs, dim, h, w)
        x = self.model(x)    
        # print('model output: ', x.shape)
        
        if not 'eva' in CFG.model_name:
            pool = F.adaptive_avg_pool2d(x, 1)
            # print('after pooling: ', pool.shape)
            reshaped = pool.reshape(batch_size, -1)  
            # print('after reshape: ', reshaped.shape)
        
        else:
            reshaped = x
        
        if self.is_training:
            logit = 0
            # print(len(self.dropout))
            for i in range(len(self.dropout)):
                dropped = self.dropout[i](reshaped)
                # print(dropped.shape)
                logit += self.target(dropped)
            logit = logit / len(self.dropout)
        else:
            logit = self.target(reshaped)
            
        output = {}
        if 'loss' in self.output_type:
            if target.dim() == 1:
                target = target.view(-1, 1)
            output['bce_loss'] = F.binary_cross_entropy_with_logits(logit.float(), target.float())

        if 'infer' in self.output_type:
            output['target'] = torch.sigmoid(logit.float())

        return output
    
    def initialize_dummy(self):
        # Initialize the DynamicLinear layer with dummy input data
        original_output_type = self.output_type
        self.output_type = ['infer']
        
        dummy_input = torch.zeros(1, 3, CFG.img_size, CFG.img_size)  # Assuming input size (1, 3, 224, 224)
        _ = self.forward(dummy_input)
        
        self.output_type = original_output_type

In [41]:
model = timmModel(
        model_name=CFG.model_name, 
        pretrained=CFG.pretrained, 
        num_classes=0, # no classification head
        in_channels=6, # RBG+HSV
        is_training=True
    )

In [43]:
import torch
import copy
from pathlib import Path

# Load the models
model_dir = Path(f'/root/Development/Kaggle/ISIC2024/main/models/experiments/exp{CFG.exp}/outputs')
model_paths = [
    model_dir / 'best_model_fold0.pth',
    model_dir / 'best_model_fold1.pth',
    model_dir / 'best_model_fold2.pth',
    model_dir / 'best_model_fold3.pth',
    model_dir / 'best_model_fold4.pth',
]

state_dicts = [torch.load(model_path) for model_path in model_paths]

# Initialize a new model with the same architecture
model_architecture = model  # Replace with your model class
new_state_dict = copy.deepcopy(state_dicts[0])

# Average the weights
for key in new_state_dict.keys():
    for state_dict in state_dicts[1:]:
        new_state_dict[key] += state_dict[key]
    new_state_dict[key] = torch.div(new_state_dict[key], len(state_dicts))

# Load the averaged weights into the new model
model.initialize_dummy()
model_architecture.load_state_dict(new_state_dict)

# Move the model to GPU
if torch.cuda.is_available():
    print('ok')
    model_architecture = model_architecture.to('cuda')

# Save the new model with averaged weights
torch.save(model_architecture.state_dict(), model_dir / 'averaged_model.pth')

# # Example inference with AMP
# def infer_with_amp(model, input_tensor):
#     model.eval()
#     if torch.cuda.is_available():
#         input_tensor = input_tensor.to('cuda')
#     with torch.no_grad():
#         with torch.cuda.amp.autocast():
#             output = model(input_tensor)
#     return output

# # Example input tensor
# example_input = torch.randn(1, 3, 224, 224)  # Replace with appropriate dimensions

# # Perform inference
# output = infer_with_amp(model_architecture, example_input)
# print(output)


/tmp/ipykernel_894951/4218267480.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dicts = [torch.load(model_path) for model_path in model_paths]


ok
